## The System:

![grafik.png][def]


[def]: .\PTG_System.png

## Preperation
#### Welche Packages brauchen wir NICHT?

In [1]:
!pip install gurobipy==10.0.0
!pip install pyomo==6.6.2
!pip install numpy==1.24.3
!pip install pandas==1.3.5
#!pip install tsam==2.3.1
!pip install matplotlib
#!pip install july

In [2]:
from gurobipy import *
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import tsam.timeseriesaggregation as tsam
#import july
#from july.utils import date_range


## Time series
#### Daten laden

In [6]:
data_sheet = pd.read_excel(".\Data_Sheet_neu.xlsx")
#print(data_sheet)

time_slices = np.array(time_series_data.index)
#print(time_slices)

      Unnamed: 0  s     t  dt  PowerD_demand  HeatD_demand  P_PV_tot
0              0  1     1   1          158.4     69.178754   0.00000
1              1  1     2   1          156.0     56.480866   0.00000
2              2  1     3   1          156.0     45.267146   0.00000
3              3  1     4   1          156.6     39.165564   0.00000
4              4  1     5   1          151.2     40.567279   0.00000
...          ... ..   ...  ..            ...           ...       ...
8755        8755  1  8756   1          168.6     15.307163   4.33854
8756        8756  1  8757   1          174.0     14.920863   0.00000
8757        8757  1  8758   1          167.4     14.293124   0.00000
8758        8758  1  8759   1          166.8     23.033177   0.00000
8759        8759  1  8760   1          160.2     32.497543   0.00000

[8760 rows x 7 columns]
[   0    1    2 ... 8757 8758 8759]


## Setting up the model:

In [7]:
model = ConcreteModel() #aus Pyomo

# declare sets
model.time_slices = set(time_slices) #was passiert hier genau? iwas mit Pyomo

In [ ]:
# declare decision variables
model.OPEX = Var(domain=NonNegativeReals) # [€]

# declare time dependent decision variables
model.P_PV = Var(model.time_slices, domain=NonNegativeReals) # Photovoltaic [kW], nötig?
model.H2_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW]
model.CO2_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kg_CO2/hr]
model.CH4_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW]
model.Q_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW]
model.P_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW]
model.Q_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [MW]

model.P_battery = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_battery = Var(model.time_slices, domain=NonNegativeReals) # [MWh]
model.P_thermalStorage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_thermalStorage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]
model.P_H2_Storage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_H2_Storage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]
model.P_CO2_Storage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_CO2_Storage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]
model.P_CH4_Storage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_CH4_Storage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]

In [ ]:
# declare parameters
#i = 0.05
#lifetime = 30
#model.RBF = ((1 + i) ** lifetime * i) / ((1 + i) ** lifetime - 1)

model.P_N_PV = 900 #[kw]
model.P_N_PEM = 900 #[kW]
model.CO2_N_MR = 1500 #[kg/hr]
model.Q_N_CHP = 800 #[kW]
model.Q_N = 500 #[kW]

#etas + storages

model.Q_max_B = 10 # [MW] maximal installable capacity of boiler
model.eta_B = 0.9
model.specific_invest_B = 2500 # [€/MW]

model.Q_max_HP = 10 # [MW] maximal installable capacity of heat pump
model.eta_HP = 3
model.specific_invest_HP = 100000 # [€/MW]

model.Q_max_storage = 40 # [MWh] maximal installable capacity of storage
model.storage_loss = 0.01 # [1/h] hourly storage loss
model.specific_invest_storage = 1000 # [€/MWh]

In [ ]:
# declare time dependent parameters
model.time_slice_weight = Param(model.time_slices, initialize = time_slice_weight)
model.gas_price = Param(model.time_slices, initialize = gas_price_time_series)
model.electricity_price = Param(model.time_slices, initialize = electricity_price_time_series)
model.Q_demand = Param(model.time_slices, initialize = heat_demand_time_series)

In [ ]:
# declare unindexed constraints



# declare rules to setup indexed constraints



# declare indexed constraints

## Solving the problem

In [ ]:
#SolverFactory('gurobi').solve(model).write()